# 데이터 불러오기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


필요한 implicit 라이브러리 다운로드하기

In [2]:
pip install implicit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.6/18.6 MB 71.3 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np

데이터 설명 
- transaction_new는 기존에 데이터에 유저세그멘테이션을 하고 그 세그멘테이션 결과를 뒤에 붙혀서 각 거래는 어떤 그룹의 사용자가 거래를 하였는지 보여줌
- m_df_p_2는 기존 거래데이터와 product데이터를 붙힌 것으로 각 데이터에 거래된 상품의 자세한 정보가 붙어있음
- rfm은 각 유저의 rfm(빈도,최근거래,거래량)을 계산하고 그 사용자가 어디에 속하는지를 보여주는 데이터 프레임(이때 거래내역이 없는 사용자는 rfm 할 수 없음)
- cb_data는 거래내역이 성공한 데이터에서 거래내역과 관련된 특성만 가져온 데이터 프레임
- cb_product2는 거래내역에서 quantity가 너무 적은 데이터는 삭제한 최종 데이터(30을 기준으로 30이하는 삭제함)
- customer_merge는 거래내역과 사용자 id 거래 시간 및 상품명이 있는 데이터 (이때 거래수량이 30이상만 포함)

In [5]:
transaction_new = pd.read_csv('/content/drive/MyDrive/fashion campus2/transaction_new4.csv')

m_df_p_2 = pd.read_csv('/content/drive/MyDrive/fashion_campus_final/m_df_p_2.csv')

rfm    = pd.read_csv('/content/drive/MyDrive/fashion campus2/rfm2.csv')

product   = pd.read_csv('/content/drive/MyDrive/fashion_campus_final/product.csv',error_bad_lines=False)


cb_data = pd.read_csv('/content/drive/MyDrive/fashion_campus_final/cb_data.csv')

customer_merge = pd.read_csv('/content/drive/MyDrive/fashion_campus_final/customer_merge.csv')

cb_product2 = pd.read_csv('/content/drive/MyDrive/fashion_campus_final/cb_product2.csv')


/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
b'Skipping line 6044: expected 10 fields, saw 11\nSkipping line 6569: expected 10 fields, saw 11\nSkipping line 7399: expected 10 fields, saw 11\nSkipping line 7939: expected 10 fields, saw 11\nSkipping line 9026: expected 10 fields, saw 11\nSkipping line 10264: expected 10 fields, saw 11\nSkipping line 10427: expected 10 fields, saw 11\nSkipping line 10905: expected 10 fields, saw 11\nSkipping line 11373: expected 10 fields, saw 11\nSkipping line 11945: expected 10 fields, saw 11\nSkipping line 14112: expected 10 fields, saw 11\nSkipping line 14532: expected 10 fields, saw 11\nSkipping line 15076: expected 10 fields, saw 12\nSkipping line 29906: expected 10 fields, saw 11\nSkipping line 31625: expected 10 fields, saw 11\nSkipping line 33020: expected 1

In [6]:
# 기존의 product에 id를 product_id로 바꿈

product.rename(columns = {'id':"product_id"},inplace=True)

In [7]:
# 위에서 읽은 transaction_new에서 그룹에 따라서 거래를 4개로 나눈다.
# 예를 들어 trans_customer는 customer에 속하는 사용자들의 거래내역을 보여준다.

trans_extinct_customer = transaction_new[transaction_new['group'] == 'extinct customer']

trans_customer = transaction_new[transaction_new['group'] == 'customer']

trans_vip = transaction_new[transaction_new['group'] == 'vip']

trans_vvip = transaction_new[transaction_new['group'] == 'vvip']

In [8]:
#그중에 중요한 특성만 따로 저장한다.

trans_extinct_customer_1 = trans_extinct_customer[['customer_id','product_id','quantity','group','product_name']]
trans_customer_1 = trans_customer[['customer_id','product_id','quantity','group','product_name']]
trans_vip_1 = trans_vip[['customer_id','product_id','quantity','group','product_name']]
trans_vvip_1 = trans_vvip[['customer_id','product_id','quantity','group','product_name']]

# 비회원의 경우 통계적시스템으로 추천해주기

In [9]:
'''
최근출시순으로 아이템을 추천해주는 통계적 시스템으로 같은 출시일의 경우에는 판매량이 많은 아이템을 먼저 추천해주는 함수
'''

def item_year(s):

  temp = cb_data[cb_data['articleType'] == s].sort_values(['year','quantity'],ascending = False).head(10)

  return temp

In [10]:
'''
할인량이 많은순으로 아이템을 추천해주는 통계적 시스템 
'''



def promote_amout(s):
  b = m_df_p_2[m_df_p_2['articleType'] == s].sort_values('promo_amount',ascending = False).head(10)

  c= product.columns.tolist()
  c.append('promo_amount')
  c.append('total_amount')

  df_promote = b[c]

  return  pd.DataFrame(df_promote)

In [11]:
'''
상품 가격이 높은순으로 아이템을 추천해주는 통계적 시스템 
'''

def item_price_top(s):
  d = m_df_p_2[m_df_p_2['articleType'] == s].sort_values('item_price',ascending = False).head(10)

  c= product.columns.tolist()
  c.append('item_price')
  df_item = d[c]

  return  pd.DataFrame(df_item)

In [12]:
'''
상품 가격이 낮은순으로 아이템을 추천해주는 통계적 시스템 
'''

def item_price_bot(s):
  d = m_df_p_2[m_df_p_2['articleType'] == s].sort_values('item_price',ascending = True).head(10)

  c= product.columns.tolist()
  c.append('item_price')
  df_item = d[c]

  return  pd.DataFrame(df_item)

In [13]:
# 총판매량에 따른 통계적 추천함수
 

def total_amout(s):



  #해당 카테고리의 상위 판매량이 높은 상위 10개 추출
  a = m_df_p_2[m_df_p_2['articleType'] == s].groupby('product_id').sum().sort_values('total_amount',ascending = False).head(10).index



  li = []

  for i in range(10):
    li.append(product[product['product_id'] == a[i]].index[0])

  df_total = product.iloc[li,:]
  return  pd.DataFrame(df_total)

In [14]:
# 판매량에 따른 추천을 해주는 함수



def recommend_product2(df, category,top=20):
    cb_data2 = df[['productDisplayName','gender','subCategory','articleType','baseColour','season','year','usage','quantity']]
    recom_data = cb_data2[cb_data2['articleType']==category].sort_values('quantity',ascending=False)[:top]
    return recom_data

In [15]:
'''
통계적 시스템에 따른 추천해주는 함수
프로그램을 실행하면 먼저 원하는 상품을 입력한다. ex) Tshirts
그러면 어떤 기준에 따라서 상품을 볼 것인지 선택하는 창이 나온다. '| 판매량순 | 할인순 | 높은가격순 | 낮은가격순 | 최근일순 |
그러면 선택한다. ex) "높은가격순"을 입력
그러면 그 상품을 원하는 기준에 따라서 보여준다 ex) 티셔츠를 높은 가격순으로 추천해줌

'''






def program():
  print("-" * 250)
  s = input('원하는 상품 입력 :')
  
  k = input('| 판매량순 | 할인순 | 높은가격순 | 낮은가격순 | 최근일순 |')

  if k == '판매량순':
    return recommend_product2(cb_data,s)
  
  elif k == '할인순':
    return promote_amout(s)

  elif k == '높은가격순':
    return item_price_top(s)
  
  elif k == '낮은가격순':
    return item_price_bot(s)

  elif k == '최근일순':
    return item_year(s)

In [16]:
# 상품 종류 확인하기 여기서 검색어를 입력하면 된다.

product['articleType'].unique()

array(['Shirts', 'Jeans', 'Watches', 'Track Pants', 'Tshirts', 'Socks',
       'Casual Shoes', 'Belts', 'Flip Flops', 'Handbags', 'Tops', 'Bra',
       'Sandals', 'Shoe Accessories', 'Sweatshirts', 'Deodorant',
       'Formal Shoes', 'Bracelet', 'Lipstick', 'Flats', 'Kurtas',
       'Waistcoat', 'Sports Shoes', 'Shorts', 'Briefs', 'Sarees',
       'Perfume and Body Mist', 'Heels', 'Sunglasses', 'Innerwear Vests',
       'Pendant', 'Nail Polish', 'Laptop Bag', 'Scarves', 'Rain Jacket',
       'Dresses', 'Night suits', 'Skirts', 'Wallets', 'Blazers', 'Ring',
       'Kurta Sets', 'Clutches', 'Shrug', 'Backpacks', 'Caps', 'Trousers',
       'Earrings', 'Camisoles', 'Boxers', 'Jewellery Set', 'Dupatta',
       'Capris', 'Lip Gloss', 'Bath Robe', 'Mufflers', 'Tunics',
       'Jackets', 'Trunk', 'Lounge Pants', 'Face Wash and Cleanser',
       'Necklace and Chains', 'Duffel Bag', 'Sports Sandals',
       'Foundation and Primer', 'Sweaters', 'Free Gifts', 'Trolley Bag',
       'Tracksuits', 'S

In [ ]:
program()

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
원하는 상품 입력 :Sunscreen
| 판매량순 | 할인순 | 높은가격순 | 낮은가격순 | 최근일순 |최근일순


,product_id,quantity,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName
1623403,57611,18,57611,Women,Personal Care,Skin Care,Sunscreen,White,Spring,2017.0,NaN,Lotus Herbals Safe Sun Absolute Anti Tan Scrub
878779,57615,13,57615,Women,Personal Care,Skin Care,Sunscreen,Orange,Spring,2017.0,Casual,Lotus Herbals UV Screen Matte Gel 50g
85665,57610,12,57610,Women,Personal Care,Skin Care,Sunscreen,White,Spring,2017.0,NaN,Lotus Herbals Safe Sun Multi-Function Sunblock
961456,57625,9,57625,Unisex,Personal Care,Skin Care,Sunscreen,Orange,Spring,2017.0,Casual,Lotus Herbals Kids Sun Block Cream
1014396,57861,9,57861,Women,Personal Care,Skin Care,Sunscreen,Green,Spring,2017.0,Casual,Biotique Women Bio Vera Ultra Soothing Body Lo...
1623404,57611,9,57611,Women,Personal Care,Skin Care,Sunscreen,White,Spring,2017.0,NaN,Lotus Herbals Safe Sun Absolute Anti Tan Scrub
1805924,57609,9,57609,Women,Personal Care,Skin Care,Sunscreen,Orange,Spring,2017.0,NaN,Lotus Herbals Safe Sun Collagenshield Sunblock...
1764614,57621,8,57621,Women,Personal Care,Skin Care,Sunscreen,Orange,Spring,2017.0,Casual,Lotus Herbals Safe Sun Anti-Tan Sunscreen
255688,57620,7,57620,Women,Personal Care,Skin Care,Sunscreen,White,Spring,2017.0,Casual,Lotus Herbals Safe Sun Daily Sunblock
546328,57867,7,57867,Women,Personal Care,Skin Care,Sunscreen,Yellow,Spring,2017.0,Casual,Biotique Women Bio Aloevera SPF 30 Sunscreen L...


# 비회원이 아니라 회원의 경우
- 회원의 경우는 먼저 회원을 rfm으로 분리한다.
- 회원을 그룹화하여서 그룹화된 고객에 따라서 추천을 해준다
- 이떄 rfm은 거래횟수, 최근거래일, 총구매액에 사분위수를 기준으로 점수를 매겨서 고객을 나눈다.
- 이렇게 해서 나눈 그룹을 transaction_new4에 붙힘 (따로 과정은 보기 길 수 있으니 그냥 read_csv만 함)

### rfm결과 각각의 사분위수 기준

빈도수 
- 0 ~3 
- 3 ~ 12 
- 12 ~ 40
- 40 ~

총금액
- 0 ~ 909450.0
- 909450.0 ~ 5144328.0
- 5144328.0 ~ 26911910.0
- 26911910.0 ~

최근구매일
- 20210810 이전
- 20210810 ~ 20220410 사이
- 20220410 ~ 20220620 사이
- 20220620 이후

In [ ]:
# 얘를 들어 4 4 4점은 vvip이고 3 3 2는 vip

rfm.head()

,Recency,Frequency,Monetary,group
0,4.0,4.0,4.0,vvip
1,3.0,3.0,2.0,vip
2,3.0,2.0,2.0,customer
3,2.0,1.0,1.0,extinct customer
4,2.0,2.0,3.0,customer


위에서 read한 customer_merge와 cb_product2를 이용해서 유사한 아이템추천해주기
- transaction에는 대부분 vvip와 vip의 거래내역만 많고 extinct와 customer의 고객은 정보가 별로 없어서 추천하기 힘들다. 
- 따라서 위 1,2 분위의 고객은 자연어처리를 이용한 모델로 추천해준다.
- 이 사용자들에게는 최근 구매한 아이템을 다른 아이템과 유사도를 구해서 비슷한 아이템을 추천해준다.

In [17]:
# userid 넣으면 해당 유저의 최근 거래 물품을 토대로 tfidf 이용하여 물품 추천

def data_similarity(df, userid,top=20):
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.metrics.pairwise import cosine_similarity

    # tfidf로 코사인 유사도를 구함
    tfidf = TfidfVectorizer()
    product_tfidf = tfidf.fit_transform(df['productDisplayName'])
    product_similarity = cosine_similarity(product_tfidf,product_tfidf)


    product_sim = {}
    for i, c in enumerate(df['productDisplayName']): 
        product_sim[i] = c

    # id와 movie title를 매핑할 dictionary를 생성
    idproduct = {}
    for i, c in product_sim.items():
        idproduct[c] = i

    product_name = customer_merge[customer_merge['customer_id']==userid]['productDisplayName'].values[0]
    
    idx = idproduct[product_name] 
    sim_scores = [(i, c) for i, c in enumerate(product_similarity[idx]) if i != idx] # 자기 자신을 제외한 유사도 및 인덱스를 추출 
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True) # 유사도가 높은 순서대로 정렬 
    sim_scores = [(product_sim[i], score) for i, score in sim_scores[0:top]]
    product_topsim = pd.DataFrame(sim_scores)
    product_topsim.columns = ['productDisplayName','similarity']
    return product_topsim

In [18]:
'''
테스트를 해보면 23071의 사용자에게 최근구매한 아이템과 유사한 아이템을 추천해주는 것을 볼 수 있다.
'''

data_similarity(cb_product2,23071)

,productDisplayName,similarity
0,ADIDAS Men Red Jersey,0.582805
1,Nike Men Black Jersey,0.492195
2,ADIDAS Men Black Jersey,0.487247
3,ADIDAS Men Blue Jersey,0.478496
4,Baggit Women Roxy Net Black Handbag,0.466709
5,Nike Men Green Jersey,0.461352
6,Baggit Women Roxy Net Brown Handbag,0.458205
7,ADIDAS Men Printed Blue Jersey,0.435937
8,ADIDAS Men Solid Black Jersey,0.426816
9,ADIDAS Men Solid Blue Jersey,0.420897


## implicit를 이용한 추천시스템
- vip, vvip의 경우 해당
- 먼저 상품을 검색하면 그 상품과 해당 사용자의 그룹원들과 user_item 행렬을 만듬
- 그 행렬을 이용해서 implicit의 als에 학습시킴
- 학습시키고 아이템을 추천해줌

In [19]:
# 상품을 입력하면 그 상품 거래내역이 있는 고객으로만 매트릭스를 만들어서 약간 걸림 
# 함수에 사용자id와 검색단어(ex Shirts) 그리고 고객세그멘티이션을 매개변수로 넣는다.

def matrix(user_id,query,group):

  if group == "extinct customer":
    temp_df = trans_extinct_customer_1
  elif group == "customer":
    temp_df = trans_customer_1
  elif group == "vip":
    temp_df = trans_vip_1
  else:
    temp_df = trans_vvip_1



  a = set(temp_df['customer_id'])
  b = set(temp_df[temp_df['product_name'] == query]['product_id'].tolist())
  temp_matrix = np.zeros((len(a),len(b)))
  
  temp_matrix = pd.DataFrame(temp_matrix)
  
  temp_matrix.index = sorted(list(a))
  temp_matrix.columns = sorted(list(b))

  user_idx = sorted(list(a)).index(user_id)


  #따로 쓸 검색어가 지정된 데이터
  shirt_df = temp_df[temp_df['product_name'] == query]

  for i in shirt_df.index:
    temp_user = shirt_df.loc[i]['customer_id']
    temp_product = shirt_df.loc[i]['product_id']
    temp_q = shirt_df.loc[i]['quantity']

    if temp_product in product.index:
      temp_matrix.loc[temp_user,temp_product] += temp_q
    
  
  temp_matrix.index = list(range(0,len(a)))

  return temp_matrix,user_idx

In [20]:
# 모델을 als로 만들고 fit하는함수
# 이때 위에서 만들어진 matrix를 넣는게 아니라 coo_matrix를 사용해서 희소행렬로 만들어서 fit해야 한다.
# 모델과 희소행렬을 리턴함


import implicit
import scipy



def model(matrix):
  temp_matrix = scipy.sparse.coo_matrix(matrix)
  model = implicit.als.AlternatingLeastSquares(factors=50,iterations=15,regularization=0.001)

  model.fit(temp_matrix)
  
  return model,temp_matrix

/usr/local/lib/python3.8/dist-packages/implicit/gpu/__init__.py:13: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: no CUDA-capable device is detected (/project/./implicit/gpu/utils.h:71)'
  warnings.warn(


In [21]:
#만든 모델로 추천해주는 함수 

def recommend_product(model,matrix,user_id):
  a,_ = model.recommend(user_id, matrix.tocsr()[user_id])

  li = []
  for i in a:
    if not product[product['product_id'] ==i].empty:
      li.append(product[product['product_id'] == i].index[0])

  result = product.loc[li,:]

  return result

위의 비회원일때의 통계적 추천시스템과 유사도 모델,implicit 모델을 총 집합한 program()2 

In [22]:
# 로그인 id를 입력하고 아닐시 비회원이라고 치면됨
# 비회원이거나 거래내역에 없으면 통계적 프로그램 program실행
# 회원이면 세그멘테이션을 통해서 고객이 어떤 그룹에 속했나 확인함


# 1,2면 자연어처리를 이용한 모델을 사용하고 3,4면 implicit를 사용한 모델로 추천을 해주자






import implicit

def program2():
  #사용자 id를 입력받음 만약 비회원이면 "비회원"이라고 입력함
  user_id = input('로그인 id를 입력하세요/비회원시 비회원 입력')
  
  #비회원이면 통계적 추천하기 program은 위에서 선언한 program함수로 자동시작함
  if user_id == '비회원':
    result = program()
    return result
  
  #회원이라면 추천해주기 
  else: 
    user_id = int(user_id)
    
    #근데 회원이라도 거래내역이 없으면 추천을 하지 못해서 거래내역이 있는 사람만 추천을 함
    if user_id in transaction_new['customer_id'].tolist():
      group = list(set(transaction_new[transaction_new['customer_id'] ==user_id ]['group']))[0]


      if group == "extinct customer" or group == "customer":

        result = data_similarity(cb_product2,user_id)
        return result

      else:

        #원하는 상품 검색하기
        query = input('검색창 : ')
        
        #vip,vvip면 svd를 사용하고 소멸고객이나 일반고객이면 자연어처리기반모델을 사용한다.
        
        #user_item_matrix = 행렬 만드는 함수
        user_item_matrix,user_idx = matrix(user_id,query,group)

        # #모델 생성 및 학습하는 함수 만들기
        model_re,temp_matrix = model(user_item_matrix)
        

        #만들어진 모델을 이용해서 추천해주기
        result = recommend_product(model_re,temp_matrix,user_idx)

        return result
    
    #회원이라도 거래내역이 없으면 통계적 시스템을 사용함
    else:
      result = program()
      return result

In [ ]:
# 비회원의 경우 추천시스템 테스트

program2()

로그인 id를 입력하세요/비회원시 비회원 입력비회원
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
원하는 상품 입력 :Socks
| 판매량순 | 할인순 | 높은가격순 | 낮은가격순 | 최근일순 |낮은가격순


,product_id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,item_price
14962,6705,Women,Accessories,Socks,Socks,White,Summer,2011.0,Sports,Nike Women's Nsw Ctn White Red Socks,14650
536680,29119,Unisex,Accessories,Socks,Socks,Black,Summer,2012.0,Casual,Puma Kids Unisex Pack of 3 Black Socks,20190
747174,17582,Unisex,Accessories,Socks,Socks,White,Fall,2011.0,Sports,ADIDAS Unisex Pack of 3 Crew Socks,22437
597387,31996,Men,Accessories,Socks,Socks,White,Summer,2016.0,Formal,Playboy Men White Socks,22815
218727,57915,Men,Accessories,Socks,Socks,Black,Summer,2016.0,Casual,Fruit of the Loom Men Black Pack of Three Socks,26423
672878,58315,Men,Accessories,Socks,Socks,White,Summer,2016.0,Sports,Jockey LCESCBRA Men White Pack of 3 Socks 2004,26943
420974,40017,Unisex,Accessories,Socks,Socks,White,Summer,2012.0,Casual,ADIDAS Unisex White Adicrew Socks,30314
84580,18952,Men,Accessories,Socks,Socks,Black,Summer,2016.0,Casual,Levis Men Fader Stripes Flat Knit Crew Pack of...,33086
898116,46878,Men,Accessories,Socks,Socks,Black,Summer,2012.0,Casual,Lino Perros Men Pack of 2 Socks,33403
622549,51151,Men,Accessories,Socks,Socks,Grey,Summer,2016.0,Casual,Raymond Men Grey Socks,34910


In [ ]:
#vip고객인 24915id를 입력하고 아이템을 입력하면 implicit를 이용해서 추천하는 모습

program2()

로그인 id를 입력하세요/비회원시 비회원 입력24915
검색창 : Tshirts


/usr/local/lib/python3.8/dist-packages/implicit/utils.py:138: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.00025653839111328125 seconds
  warnings.warn(


  0%|          | 0/15 [00:00<?, ?it/s]

,product_id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName
14159,2324,Men,Apparel,Topwear,Tshirts,Black,Fall,2010.0,Casual,Reebok Men Sleeveless Black T-shirt
37685,2744,Girls,Apparel,Topwear,Tops,Orange,Summer,2011.0,Casual,Disney Kids Girl's Orange Stripe Kidswear
20856,4366,Men,Apparel,Topwear,Tshirts,Black,Summer,2011.0,Casual,Pink Floyd Men's The Wall Black T-shirt
34441,3265,Men,Apparel,Topwear,Tshirts,Blue,Summer,2011.0,Casual,Guerrilla Men's Kill You Blue T-shirt
43360,6033,Men,Apparel,Topwear,Tshirts,Black,Summer,2011.0,Casual,UCB Men's VNeck With Contrast Logo Black T-shirt
28060,3649,Men,Footwear,Shoes,Casual Shoes,Black,Fall,2011.0,Casual,FILA Men Elite Black Casual Shoes


In [ ]:
# customer인 99645 id를 입력하면 자연어처리모델로 추천을 해주는 모습

program2()

로그인 id를 입력하세요/비회원시 비회원 입력99645


,productDisplayName,similarity
0,Numero Uno Men Casual White Shoes,1.000000
1,Numero Uno Men Casual White Casual Shoes,0.960772
2,Numero Uno Men White Shoes,0.939079
3,Numero Uno Men's Casual Black Shoes,0.929523
4,Numero Uno Men Black Casual Shoes,0.929523
5,Numero Uno Men Casual Black Shoes,0.929523
6,Numero Uno Men's Casual White Shoe,0.907813
7,Numero Uno Men's Casual Grey Shoes,0.901544
8,Numero Uno Men Brown Casual Shoes,0.899751
9,Numero Uno Men Casual Brown Shoes,0.899751
